In [120]:
# Normal approximation to binomial distribution

from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n, p)"""
    
    mu = p * n
    sigma = math.sqrt(p * (1-p) * n)
    
    return mu, sigma

In [121]:
%%capture
%run ch6_probability.ipynb

In [122]:
# Normal probability

# The normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# It's above the threshold if it's not below the threshold
def normal_probability_above(lo: float, 
                             mu: float = 0, 
                             sigma: float = 1) -> float:
    
    """The probability that an N(mu, sigma) is greater than lo."""
    
    return 1 - normal_cdf(lo, mu, sigma)

# It's between if it's less than hi, but not less than lo
def normal_probability_between(lo: float, 
                               hi: float, 
                               mu: float = 0, 
                               sigma: float = 1) -> float:
    
    """The probability that an N(mu, sigma) is between lo and hi."""
    
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo: float, 
                               hi: float, 
                               mu: float = 0, 
                               sigma: float = 1) -> float:
    
    """The probability that an N(mu, sigma) is not between lo and hi."""
    
    return 1 - normal_probability_between(lo, hi, mu, sigma)    

In [123]:
# Normal bound

def normal_upper_bound(p: float, 
                       mu: float = 0, 
                       sigma: float = 1) -> float:
    
    """Returns the z for which P(Z <= z) == probability"""
    
    return inverse_normal_cdf(p, mu, sigma)

def normal_lower_bound(p: float, 
                       mu: float = 0, 
                       sigma: float = 1) -> float:
    
    """Returns the z for which P(Z >= z) == probability"""
    
    return inverse_normal_cdf(1 - p, mu, sigma)

def normal_two_sided_bounds(p: float, 
                            mu: float = 0, 
                            sigma: float = 1) -> Tuple[float, float]:
    
    """
    Returns the symmetric (about the mean) bounds 
    that contain the specified probability
    """
    
    tail_probability = (1 - p) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [124]:
# Example: flip the coin 1000 times (null hypothesis is true)

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

print(mu_0) # 1000 * 0.5 = 500
print(sigma_0) # sqrt((1000 * 0.5) * 0.5) = 15.8

500.0
15.811388300841896


In [125]:
# Significance of a test

# We are 95% confident that null hypothesis is true
# There is 5% chance that we reject the null hypothesis even though it is true (type I error; false positive)

lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

print(lower_bound)
print(upper_bound)

469.01026640487555
530.9897335951244


In [126]:
# Power of a test

# 95% bounds based on assumption p = 0.5 (null hypothesis is true)
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p = 0.55 (biased toward heads)
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# type II error (false negative) means we fail to reject the null hypothesis even though it is false
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

# probability of not making type II error
power = 1 - type_2_probability

print(power)

0.8865480012953671


In [127]:
# One-sided test

# 95% bounds based on assumption p <= 0.5 (not biased toward heads)
hi = normal_upper_bound(0.95, mu_0, sigma_0)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

print(power)

0.9363794803307173


In [ ]:
# p-values
# Probability assuming null hypothesis is true

def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    How likely are we to see a value at least as extreme as x (in either direction)
    if our values are from an N(mu, sigma)?
    """
    
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        
        return 2 * normal_probability_below(x, mu, sigma)

print(two_sided_p_value(529.5, mu_0, sigma_0))

In [ ]:
import random

extreme_value_count = 0

for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 
                    for _ in range(1000))
    
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
        
# p-value was 0.062 => ~62 extreme values out of 1000 trials
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

In [ ]:
print(two_sided_p_value(531.5, mu_0, sigma_0))

In [ ]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

print(upper_p_value(524.5, mu_0, sigma_0))
print(upper_p_value(526.5, mu_0, sigma_0))

In [ ]:
# Confidence interval

# We observe 525 heads out of 1000 flips (estimated p = 0.525)
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

print(mu)
print(sigma)

In [ ]:
print(normal_two_sided_bounds(0.95, mu, sigma))

In [ ]:
p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(normal_two_sided_bounds(0.95, mu, sigma))

In [ ]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert num_rejections == 46

In [ ]:
# A/B test

def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N # probability that someone clicks ad
    sigma = math.sqrt(p * (1 - p) / N)
    
    return p, sigma

# Test null hypothesis (p_A and p_B are the same)

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [ ]:
z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

In [ ]:
print(two_sided_p_value(z)) # don't reject the null hypothesis (no difference)

In [ ]:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(two_sided_p_value(z)) # reject the null hypothesis (difference)

In [ ]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)